In [20]:
## import Library ##
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import keras
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D,Lambda, Dropout, InputLayer, Input
from keras.applications.nasnet import NASNetLarge, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.callbacks import EarlyStopping, LearningRateScheduler

import warnings
warnings.filterwarnings('ignore')

In [2]:
## Load Dataset ##
train_dir = '../input/dog-breed-identification/train'
test_dir = '../input/dog-breed-identification/test'

In [3]:
train_size = len(os.listdir('../input/dog-breed-identification/train'))
test_size = len(os.listdir('../input/dog-breed-identification/test'))

train_size,test_size

In [4]:
df = pd.read_csv('../input/dog-breed-identification/labels.csv')
df.head()

In [5]:
train_df = pd.read_csv("../input/dog-breed-identification/labels.csv")
train_df['id'] = train_df['id'].apply(lambda x: x + '.jpg')
train_df.head()

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255, 
                                 validation_split=0.2,
                                 horizontal_flip=True)
img_size = (331,331,3)

train_generator=train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/dog-breed-identification/train/",
    x_col="id",
    y_col="breed",
    batch_size=32,
    class_mode="categorical",
    target_size=img_size,
    subset='training')

valid_generator=train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/dog-breed-identification/train/",
    x_col="id",
    y_col="breed",
    batch_size=32,
    class_mode="categorical",    
    target_size=img_size,
    subset='validation')

In [7]:
#Extracting different classes
dog_breeds = sorted(df['breed'].unique())
n_classes = len(dog_breeds) 
print(n_classes)

In [8]:
# grouped = df.groupby('breed')
# grouped.count().sort_values(['id'], ascending=False)

In [9]:
## convert classes to num ##
class_to_num = dict(zip(dog_breeds,range(n_classes)))

In [10]:
def images_to_array(data_dir,df,image_size):
    image_names = df['id']
    image_labels = df['breed']
    data_size = len(image_names)
    
    X = np.zeros([data_size,image_size[0],image_size[1],image_size[2]],dtype = np.uint8)
    y = np.zeros([data_size,1],dtype = np.uint8)
    
    for i in range(data_size):
        img_name = image_names[i]
        img_dir = os.path.join(data_dir,img_name+'.jpg')
        img_pixels = load_img(img_dir,target_size=image_size)
        X[i] = img_pixels
        y[i] = class_to_num[image_labels[i]]
        
    y = to_categorical(y)
    
    ind = np.random.permutation(data_size)
    X = X[ind]
    y = y[ind]
    print('Ouptut Data Size: ', X.shape)
    print('Ouptut Label Size: ', y.shape)
    return X, y     

In [11]:
img_size = (331,331,3)
X, y = images_to_array(train_dir,df,img_size)

### Extracting Features

In [12]:
## Extracting Features ##
def get_features(model_name, data_preprocessor, input_size, data):
    #Prepare pipeline.
    input_layer = Input(input_size)
    preprocessor = Lambda(data_preprocessor)(input_layer)
    base_model = model_name(weights='imagenet', include_top=False,
                            input_shape=input_size)(preprocessor)
    avg = GlobalAveragePooling2D()(base_model)
    feature_extractor = Model(inputs = input_layer, outputs = avg)
    #Extract feature.
    feature_maps = feature_extractor.predict(data, batch_size=32, verbose=1)
    print('Feature maps shape: ', feature_maps.shape)
    return feature_maps

In [13]:
## Model (NasnetLarge) ##
nasnet_preprocessor = preprocess_input
nasnet_features = get_features(NASNetLarge, 
                              nasnet_preprocessor,
                              img_size, X)

In [14]:
## Model(InceptionResnetV2) ##
inc_resnet_preprocessor = preprocess_input
inc_resnet_features = get_features(InceptionResNetV2,
                                   inc_resnet_preprocessor,
                                   img_size, X)

In [15]:
## concatinating features ##
final_features = np.concatenate([nasnet_features, 
                                 inc_resnet_features,
                                ], axis=-1)

In [16]:
del X

In [17]:
## Building Model ##
from keras.models import Sequential
model = Sequential()
model.add(InputLayer(final_features.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(120,activation='softmax'))

In [18]:
## Callback ##
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def scheduler(epoch):
    if epoch<10:
        return 0.001
    else:
        return 0.001 * tf.math.exp(0.1 *(10-epoch))
lr_scheduler = LearningRateScheduler(scheduler)

my_callback=[EarlyStop_callback, lr_scheduler]

In [21]:
## Training Model ##
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
model.compile(optimizer= optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy'])

model.fit(final_features,
          y,
          batch_size=32,
          epochs=50,
          validation_split=0.2,
          callbacks=my_callback)

In [22]:
def images_to_array2(data_dir,df, img_size):
    images_names = df['id']
    data_size = len(images_names)
    X = np.zeros([data_size, img_size[0], img_size[1], 3], dtype=np.uint8)
    
    for i in range(data_size):
        image_name = images_names[i]
        img_dir = os.path.join(data_dir, image_name+'.jpg')
        img_pixels = load_img(img_dir, target_size=img_size)
        X[i] = img_pixels
        
    print('Ouptut Data Size: ', X.shape)
    return X

In [23]:
## Load Test Dataset ##
sample_df = pd.read_csv('../input/dog-breed-identification/sample_submission.csv')
test_data = images_to_array2(test_dir, sample_df, img_size)

In [24]:
## Extracting test features
nasnet_features = get_features(NASNetLarge, nasnet_preprocessor, img_size, test_data)
inc_resnet_features = get_features(InceptionResNetV2, inc_resnet_preprocessor, img_size, test_data)

In [25]:
test_features = np.concatenate([nasnet_features,
                                 inc_resnet_features
                               ],axis=-1)
print('Final feature maps shape', test_features.shape)

In [26]:
del test_data

In [27]:
## Evaluation ##
y_pred = model.predict(test_features, batch_size=32)

In [28]:
## Submission File ##
for breed in dog_breeds:
    sample_df[breed] = y_pred[:,class_to_num[breed]]
sample_df.to_csv('pred.csv', index=None)

In [29]:
sample_df.head()